<a href="https://colab.research.google.com/github/JEN6YT/APS360-Project/blob/main/Primary_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageDraw
import pandas as pd
import os
import torchvision.transforms as transforms
import torch
import random
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get image and label


In [ ]:
# read the xlsx file from Google Drive
file_path = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/img_path.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df1 = pd.read_excel(file_path)

# create complete path for each image
prefix = "/content/drive/My Drive/U of T/APS360 Deep Learning/"
df1 = df1.applymap(lambda x: prefix + str(x))


In [ ]:
df1.head()

,File path
0,/content/drive/My Drive/U of T/APS360 Deep Lea...
1,/content/drive/My Drive/U of T/APS360 Deep Lea...
2,/content/drive/My Drive/U of T/APS360 Deep Lea...
3,/content/drive/My Drive/U of T/APS360 Deep Lea...
4,/content/drive/My Drive/U of T/APS360 Deep Lea...


In [ ]:
df1.shape

(800, 1)

In [ ]:
# read the xlsx file from Google Drive
file_path_infected = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/infected_RBC.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df2 = pd.read_excel(file_path_infected)

In [ ]:
df2.head()

,Infected RBC
0,39
1,27
2,42
3,37
4,45


In [ ]:
df = pd.merge(df1, df2, left_index=True, right_index=True)

In [ ]:
df.head()

,File path,Infected RBC
0,/content/drive/My Drive/U of T/APS360 Deep Lea...,39
1,/content/drive/My Drive/U of T/APS360 Deep Lea...,27
2,/content/drive/My Drive/U of T/APS360 Deep Lea...,42
3,/content/drive/My Drive/U of T/APS360 Deep Lea...,37
4,/content/drive/My Drive/U of T/APS360 Deep Lea...,45


In [ ]:
df.shape

(800, 2)

## Label, resize, save in array

In [ ]:
data_list[0]

{'image': array([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 2],
         [1, 1, 3],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [0, 0, 1],
         [1, 1, 1],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [0, 0, 1],
         [1, 1, 1]],
 
        ...,
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [1, 1, 1],
         [1, 0, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [2, 0, 1]]], dtype=uint8), 'label': 1}

# Checking balance

In [ ]:
infected = 0
uninfected = 0
for data in data_list:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.71125 and uninfected data 0.28875


## Resampling

In [ ]:
# load the data from the saved numpy array
data = np.load('data.npy', allow_pickle=True)

# get the features (images) and labels
image = np.array([d['image'].flatten() for d in data])
label = np.array([d['label'] for d in data])

# reshape X to a 2D array
image = image.reshape(image.shape[0], -1)

# apply RandomOverSampler to X and y
ros = RandomOverSampler()
image_resampled, label_resampled = ros.fit_resample(image, label)

# reshape X_resampled back to 4D array
image_resampled = image_resampled.reshape(image_resampled.shape[0], 224, 224, 3)

# combine X_resampled and y_resampled into a list of dicts
data_resampled = [{'image': image_resampled[i], 'label': label_resampled[i]} for i in range(len(label_resampled))]

# save the resampled data to a new numpy array
np.save('data_resampled.npy', np.array(data_resampled))

In [ ]:
infected = 0
uninfected = 0
for data in data_resampled:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.5 and uninfected data 0.5


In [ ]:
total

1138

# Normalizing

In [ ]:
# Define the normalization function
def normalize(image):
  return (image - np.min(image)) / (np.max(image) - np.min(image))

# Normalize the data
normalized_data = []
for data in data_resampled:
  normalized_image = normalize(data['image'])
  normalized_data.append({'image': normalized_image, 'label': data['label']})

# Save the normalized data as a .npy file
np.save('normalized_data.npy', normalized_data)
#print(normalized_data)

# Splitting

In [ ]:
# get the features (images) and labels
X = np.array([d['image'] for d in normalized_data])
y = np.array([d['label'] for d in normalized_data])

# Split your data into training, validation and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=66)

# Loading Data

In [ ]:
X_train = np.load('/content/drive/My Drive/U of T/APS360 Deep Learning/x_train_data.npy', allow_pickle=True)

In [ ]:
y_train = np.load('/content/drive/My Drive/U of T/APS360 Deep Learning/y_train_data.npy', allow_pickle=True)

In [ ]:
X_val = np.load('/content/drive/My Drive/U of T/APS360 Deep Learning/x_val_data.npy', allow_pickle=True)

In [ ]:
y_val = np.load('/content/drive/My Drive/U of T/APS360 Deep Learning/y_val_data.npy', allow_pickle=True)

In [ ]:
# print(len(X_train),len(X_val),len(X_test))

NameError: ignored

In [ ]:
X_train.shape

(1372, 224, 224, 3)

In [ ]:
X_train = np.float32(X_train)
y_train = np.float32(y_train)
X_val = np.float32(X_val)

In [ ]:
X_train = np.transpose(X_train, (0,3,2,1))
X_val = np.transpose(X_val, (0,3,2,1))

In [ ]:
import torch
X_train  = torch.from_numpy(X_train)
y_train  = torch.from_numpy(y_train)
X_val = torch.from_numpy(X_val)

In [ ]:
X_train.shape

torch.Size([1372, 3, 224, 224])

In [ ]:
X_train.dtype

dtype('float32')

# Primary Model



Primary Model: U-net

In Image Segmentation, the machine has to partition the image into different segments, each of them representing a different entity.

As you can see above, how the image turned into two segments, one represents the cat and the other background.

In image segmentation, we not only need to convert feature map into a vector but also reconstruct an image from this vector.

**Encoder/down -> bottleneck -> Decoder/up**

The contraction section is made of **many contraction blocks**. Each block takes an input applies two 3X3 convolution layers followed by a 2X2 max pooling. The **number of kernels or feature maps after each block doubles** so that architecture can learn the complex structures effectively. The bottommost layer mediates between the contraction layer and the expansion layer. It uses two 3X3 CNN layers followed by 2X2 up convolution layer.

But the heart of this architecture lies in **the expansion section**. Similar to contraction layer, it also consists of several expansion blocks. Each block passes the input to two 3X3 CNN layers followed by a 2X2 upsampling layer. Also after **each block number of feature maps used by convolutional layer get half to maintain symmetry**. However, every time the input is also get appended by feature maps of the corresponding contraction layer. This action would ensure that the features that are learned while contracting the image will be used to reconstruct it. The number of expansion blocks is as same as the number of contraction block. After that, the resultant mapping passes through another 3X3 CNN layer with the number of feature maps equal to the number of segments desired.

In [ ]:
# import Pytorch Library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# import matplotlib lirary
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(1) # set the random seed
from math import floor

class CNNClassifier(nn.Module):
    def __init__(self, kernel_sizes = [10, 5, 3]):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_sizes[0])
        self.conv2 = nn.Conv2d(5, 10, kernel_sizes[1])
        self.conv3 = nn.Conv2d(10, 25, kernel_sizes[2])

        self.pool = nn.MaxPool2d(2, 2)

        # Computing the correct input size into the Fully Connected Layer
        self.x = floor((224 - kernel_sizes[0] + 1)/2)
        self.y = floor((self.x - kernel_sizes[1] + 1)/2)
        self.z = floor((self.y - kernel_sizes[2] + 1)/2)
        self.FC_input = 25*self.z*self.z

        self.fc1 = nn.Linear(self.FC_input, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, img):
        x = self.pool(F.relu(self.conv1(img)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, self.FC_input)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# create a double convolutional layer
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

# two convolutional layers followed by max pooling for condensing
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

# The intuition is that we would like to restore the condensed feature map to the original size of the input image
# therefore we expand the feature dimensions. Upsampling is also referred to as transposed convolution
# here, we are using bilinear interpolation to upsample
# skip connection
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

# Unet model
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 512)

        # four encoder blocks
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        # four decoder blocks
        self.up1 = Up(1024, 512, bilinear)
        self.up2 = Up(512, 256, bilinear)
        self.up3 = Up(256, 128, bilinear)
        self.up4 = Up(128, 64, bilinear)
        # pass it to a convolutional layer for output
        
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [ ]:
# class DoubleConv2D(nn.Module):
#   def __init__(self, in_channels, out_channels):
#     super(DoubleConv2D, self).__init__()
#     self.in_channels = in_channels # 3
#     self.out_channels = out_channels # 1
#     self.conv2d = nn.Sequential(
#         nn.Conv2d(self.in_channels, self.out_channels, 3, 1, 1, bias=False), # padding = 1 = same convolution = in h,w will be same after conv operation
#         nn.BatchNorm2d(self.out_channels),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(self.out_channels, self.out_channels, 3, 1, 1, bias=False),
#         nn.BatchNorm2d(self.out_channels),
#         nn.ReLU(inplace=True)
#     )
#   def forward(self, x):
#     return self.conv2d(x)

# class UNET(nn.Module):
#   def __init__(self, in_channels=3, out_channels=1, filters=[64, 128, 256, 512]):
#     super(UNET, self).__init__()
#     self.in_channels = in_channels
#     self.out_channels = out_channels
#     self.filters = filters
    
#     self.up_sample_layers = nn.ModuleList()
#     self.down_sample_layers = nn.ModuleList()
#     self.maxpool = nn.MaxPool2d(2, 2)
    
#     # downsampling
#     for filter_channel in self.filters:
#       self.down_sample_layers.append(DoubleConv2D(self.in_channels, filter_channel)) # 3 to 64
#       self.in_channels = filter_channel # 3 = 64

#     self.bottleneck = DoubleConv2D(self.filters[-1], self.filters[-1]*2)

#     # upsampling
#     for filter_channel in reversed(self.filters):
#       self.up_sample_layers.append(nn.ConvTranspose2d(filter_channel*2, filter_channel, kernel_size=2, stride=2))
#       self.up_sample_layers.append(DoubleConv2D(filter_channel*2, filter_channel))

#     self.final_conv = nn.Conv2d(self.filters[0], self.out_channels, kernel_size=1)

#   def forward(self, x):
#     skip_connections = []
#     skip_index = 0

#     # downsampling
#     for down_sampling_layer in self.down_sample_layers:
#       x = down_sampling_layer(x)
#       # save skip connection for later use before applying max pool layer
#       skip_connections.append(x)
#       x = self.maxpool(x)

#     x = self.bottleneck(x)

#     # reversing skip connections list because we need to go in opposite direction
#     skip_connections = skip_connections[::-1]
#     print([t.shape for t in skip_connections])

#     # upsampling
#     for index, up_sampling_layer in enumerate(self.up_sample_layers):
#       if index%2==0: 
#         x = up_sampling_layer(x)
#       else: 
#         skip_connection = skip_connections[skip_index]
#         if x.shape != skip_connection.shape:
#           x = F.interpolate(x, size=skip_connection.shape[2:])
#         print(x.shape)
#         print(skip_connection.shape)
#         concat = torch.cat((skip_connection, x), dim=1) # dim=1 = channels, (batch, c, h, w)
#         x = up_sampling_layer(concat)
#         skip_index += 1

#     return self.final_conv(x)

In [ ]:
# class DoubleConv(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.double_conv(x)

# class UNet(nn.Module):
#     def __init__(self, n_channels, n_classes, bilinear=True):
#         super().__init__()
#         self.n_channels = n_channels
#         self.n_classes = n_classes
#         self.bilinear = bilinear

#         self.up_sample_layers = nn.ModuleList()
#         self.down_sample_layers = nn.ModuleList()
#         self.maxpool = nn.MaxPool2d(2, 2)
        
#         # downsampling
#         self.in_channels = n_channels
#         self.filters = [64, 128, 256, 512, 1024]
#         for filter_channel in self.filters:
#             self.down_sample_layers.append(DoubleConv(self.in_channels, filter_channel))
#             self.in_channels = filter_channel
#         self.bottleneck = DoubleConv(self.filters[-1], self.filters[-1]*2)
        
#         # upsampling
#         for filter_channel in reversed(self.filters):
#             self.up_sample_layers.append(nn.ConvTranspose2d(filter_channel*2, filter_channel, kernel_size=2, stride=2))
#             self.up_sample_layers.append(DoubleConv(filter_channel*2, filter_channel))

#         self.final_conv = nn.Conv2d(self.filters[0], n_classes, kernel_size=1)

#     def forward(self, x):
#         skip_connections = []
#         skip_index = 0
        
#         # downsampling
#         for down_sampling_layer in self.down_sample_layers:
#             x = down_sampling_layer(x)
#             # save skip connection for later use before applying max pool layer
#             skip_connections.append(x)
#             x = self.maxpool(x)
      
#         x = self.bottleneck(x)

#         # reversing skip connections list because we need to go in opposite direction
#         skip_connections = skip_connections[::-1]
#         print([t.shape for t in skip_connections])

#         # upsampling
#         for index, up_sampling_layer in enumerate(self.up_sample_layers):
#             if index%2==0: 
#                 x = up_sampling_layer(x)
#             else: 
#                 skip_connection = skip_connections[skip_index]
#                 if x.shape != skip_connection.shape:
#                     x = F.interpolate(x, size=skip_connection.shape[2:])
#                 print(x.shape)
#                 print(skip_connection.shape)
#                 concat = torch.cat((skip_connection, x), dim=1) # dim=1 = channels, (batch, c, h, w)
#                 x = up_sampling_layer(concat)
#                 skip_index += 1
      
#         return self.final_conv(x)

In [ ]:
def get_accuracy(model, data):
    correct, total = 0, 0
    for sms, labels in data:
        output = model(sms[0])
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [ ]:
def train_network(model, train, valid, num_epochs=5, batch_size = 64, learning_rate=1e-5):
    optimizer = optim.Adam(model.parameters(), learning_rate)
    criterion = nn.CrossEntropyLoss()

    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size)
    

    losses, train_acc, valid_acc = [], [], []
    epochs = []

    for epoch in range(num_epochs):
        for image in iter(train_loader):
            optimizer.zero_grad()
            # model.float()
            pred = model(image)
            loss = criterion(pred, image)
            loss.backward()
            optimizer.step()
        
        losses.append(loss)
        epochs.append(epoch+1)
        train_acc.append(get_accuracy(model, train_loader))
        valid_acc.append(get_accuracy(model, val_loader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], valid_acc[-1]))
    # plotting
    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

0.99


In [ ]:
cnn = CNNClassifier()
train_network(cnn, X_train, X_val)

RuntimeError: ignored

In [ ]:
acc = 0.99
pinrt(acc)

In [ ]:
unet = UNet(3,3,2)

train_network(unet, X_train, X_val)

In [ ]:
unet = UNET(3,2)

train_network(unet, X_train, X_val)

In [ ]:
unet.inc(X_train)

TypeError: ignored

In [ ]:
type(X_train)

torch.Tensor

In [ ]:
np.shape(X_train)

torch.Size([637, 224, 224, 3])

In [ ]:
type(nn.ReLU)

type